# Numpy vectorizing

In [ ]:
import numpy as np

In [ ]:
def root(a, b, c):
    D = b ** 2 - 4 * a * c
    if D < 0:
        return np.nan, np.nan
    x1 = (-b + np.sqrt(D)) / (2 * a)
    x2 = (-b - np.sqrt(D)) / (2 * a)
    return x1, x2
root(1, 3, 4), root(-1, 3, 4), root(1, 0, 0)

In [ ]:
a = np.random.randn(int(1e6))
b = np.random.randn(int(1e6))
c = np.random.randn(int(1e6))
root(a, b, c)

In [ ]:
%%time
np.array([root(av, bv, cv) for av, bv, cv in zip(a, b, c)])

In [ ]:
root_vec = np.vectorize(root)
np.vectorize?

In [ ]:
%%time
root_vec(a, b, c)

In [ ]:
%%time
root_vec(a, b, 7)

# Numba

In [ ]:
import numba

In [ ]:
root_jit = numba.jit(root)

In [ ]:
%timeit root(23, 78, 19.0)

In [ ]:
%timeit root_jit(23, 78, 19.0)

In [ ]:
%%time
root_jit(a, b, 7)

In [ ]:
root_jit_vec = np.vectorize(root_jit)
%time root_jit_vec(a, b, 7)

In [ ]:
@np.vectorize
@numba.jit
def fast_root(a, b, c):
    D = b ** 2 - 4 * a * c
    if D < 0:
        return np.nan, np.nan
    x1 = (-b + np.sqrt(D)) / (2 * a)
    x2 = (-b - np.sqrt(D)) / (2 * a)
    return x1, x2
%time fast_root(a, b, 7)

In [ ]:
@numba.vectorize
def fast_root(a, b, c):
    D = b ** 2 - 4 * a * c
    if D < 0:
        return np.nan, np.nan
    x1 = (-b + np.sqrt(D)) / (2 * a)
    x2 = (-b - np.sqrt(D)) / (2 * a)
    return x1, x2
%time fast_root(a, b, 7)

# But run the profiler first

In [ ]:
def fib(n):
    if n <= 1:
        return 1
    else:
        return fib(n-2) + fib(n-1)
%time fib(33)

In [ ]:
@numba.jit
def fib_jit(n):
    if n <= 1:
        return 1
    else:
        return fib_jit(n-2) + fib_jit(n-1)
%time fib_jit(33)

In [ ]:
%time fib_jit(41)

In [ ]:
%prun fib_jit(41)

In [ ]:
%prun fib(33)

In [ ]:
from functools import lru_cache
@lru_cache(None)
def fib_cache(n):
    if n <= 1:
        return 1
    else:
        return fib_cache(n-2) + fib_cache(n-1)
%time fib_cache(33)

In [ ]:
%time fib_cache(500)

# Optimizing numpy algebra

Multiplying/adding arrays incurs a lot of overhead, because lots of intermediate arrays get created and discarded again

In [ ]:
a = np.random.randn(int(1e7))
b = np.random.randn(int(1e7))
%timeit a * b + 4.1 * a < 3 * b

In [ ]:
import numexpr as ne
%timeit ne.evaluate('a * b + 4.1 * a < 3 * b')

Efficiency gain reduces once the operations become more complex

In [ ]:
c = np.random.randn(int(1e7))
%timeit (-b - np.sqrt(b ** 2 - 4 * a * c) ) / (2 * a)

In [ ]:
%timeit ne.evaluate('(-b - sqrt(b ** 2 - 4 * a * c) ) / (2 * a)')

# Cython

http://docs.cython.org/en/latest/

Compiles most python programs to C code for extra speed (but less introspection). Also allows direct calling of C library code.

Can include python classes and much more.

In [ ]:
%load_ext cython

In [ ]:
def primes(kmax):
    p = {}
    result = []
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result
print(primes(10))
%time _ = primes(10000)

In [ ]:
%%cython -a
def cprimes(kmax):
    p = {}
    result = []
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [ ]:
print(cprimes(10))
%time _ = cprimes(10000)

## Defining C-style function

In [ ]:
%%cython -a
def cfib(n):
    if n <= 1:
        return 1
    else:
        return cfib(n-2) + cfib(n-1)

In [ ]:
%time cfib(33)

In [ ]:
%time fib_jit(33)

In [ ]:
%%cython -a
cdef int cfib(int n) nogil:
    if n <= 1:
        return 1
    else:
        return cfib(n-2) + cfib(n-1)

def fib(n):
    return cfib(n)

In [ ]:
%time fib(33)

## Cython in production code

In [ ]:
def root(a, b, c):
    D = b ** 2 - 4 * a * c
    if D < 0:
        return np.nan, np.nan
    x1 = (-b + np.sqrt(D)) / (2 * a)
    x2 = (-b - np.sqrt(D)) / (2 * a)
    return x1, x2
root(1, 3, 4), root(-1, 3, 4), root(1, 0, 0)

In [ ]:
a = np.random.randn(int(1e6))
b = np.random.randn(int(1e6))
c = np.random.randn(int(1e6))
%time np.vectorize(root)(a, b, 7)

In [ ]:
%time root_jit_vec(a, b, 7)

In [ ]:
%%cython -a
from libc.math cimport sqrt, NAN

def root(float a, float b, float c):
    cdef float D, x1, x2
    D = b ** 2 - 4 * a * c
    if D < 0:
        return NAN, NAN
    x1 = (-b + sqrt(D)) / (2 * a)
    x2 = (-b - sqrt(D)) / (2 * a)
    return x1, x2
print(root(1, 3, 4), root(-1, 3, 4), root(1, 0, 0))

## More cython

http://docs.cython.org/en/latest/src/tutorial/cython_tutorial.html

# Steps to optimizing code:
- Profile where the bottleneck is (%prun)
  - You can install a [line profiler](https://github.com/rkern/line_profiler) (available as %lprun after installation)
- Is there a faster algorithm for the bottleneck?
- If the bottleneck is vectorized: can we optimize with numexpr?
- If the internal part of the loop can not be vectorized:
    - numba jit if simple enough otherwise cython
    - the loop itself can be sped up with numpy.vectorize
- If the bottleneck gets called too often: cache the result
- If even that fails: pycuda
- If that is too slow: learn to optimize cuda code or find an easier problem